<a href="https://colab.research.google.com/github/RizkyWidodo-project/EJ-EONC_Project/blob/main/Request_Travel_Time_Google_Maps_RouteAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prepare Workspace

In [ ]:
!pip install requests pandas folium googlemaps matplotlib

import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import pytz
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# Load filtered data (make sure to upload the CSV to your Google Colab environment)
data = pd.read_csv('filename.csv', sep=';')

print("filename.csv")
print(data.columns.tolist())


#Travel Time Estimation Request

In [ ]:
# Google Maps API Key (replace with your own key)
API_KEY = 'YOUR API'

# Set the departure time explicitly: Next Monday 7 AM WIB
jakarta_tz = pytz.timezone('Asia/Jakarta')
now = datetime.now(jakarta_tz)

# Calculate next Monday at 7 AM
days_until_monday = (7 - now.weekday()) % 7
# If today is Monday and it's already past 7 AM, set it for next week Monday
if days_until_monday == 0 and now.hour >= 7:
    days_until_monday = 7

next_monday_7am = now + timedelta(days=days_until_monday)
departure_time = next_monday_7am.replace(hour=7, minute=0, second=0, microsecond=0)

# Ensure departure_time is in the future relative to 'now'
# This double-check handles edge cases where the calculation might still result in a past time
if departure_time <= now:
     departure_time += timedelta(days=7)


# Format the departure time as an ISO 8601 string with timezone offset
# The API expects this format for future timestamps
departure_timestamp = departure_time.astimezone(pytz.utc).isoformat()


# Corrected and explicitly verified Field Mask
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "routes.duration,routes.distanceMeters"
}

# Robust API function modified to accept travel_mode
def get_travel_time_distance(origin_coords, destination_coords, departure_time_iso, travel_mode):
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"

    body = {
        "origin": {"location": {"latLng": origin_coords}},
        "destination": {"location": {"latLng": destination_coords}},
        "travelMode": travel_mode
    }

    # Add traffic-awareness and departure time ONLY for DRIVE and TWO_WHEELER
    # These fields are not supported or may cause errors for WALK and BICYCLE
    if travel_mode in ["DRIVE", "TWO_WHEELER"]:
        body["routingPreference"] = "TRAFFIC_AWARE"
        # Ensure departure_time_iso is correctly formatted UTC string or includes timezone offset
        body["departureTime"] = departure_time_iso

    try:
        response = requests.post(url, headers=headers, json=body)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4XX or 5XX)
        result = response.json()

        if 'routes' in result and len(result['routes']) > 0:
            route = result['routes'][0]
            duration_str = route.get('duration')
            distance_val = route.get('distanceMeters')
            travel_time_minutes = None
            if duration_str:
                try:
                    # Duration is in seconds, e.g., "3600s"
                    duration_sec = int(duration_str.replace('s', ''))
                    travel_time_minutes = duration_sec / 60
                except ValueError:
                    print(f"⚠️ Could not parse duration: {duration_str} for mode {travel_mode}")
                    travel_time_minutes = None

            if travel_time_minutes is None or distance_val is None:
                print(f"⚠️ Missing duration or distance in API response for {travel_mode}. Origin: {origin_coords}, Dest: {dest_coords}")
                return None, None
            return travel_time_minutes, distance_val
        else:
            # This case includes when result is {} (no route found)
            # print(f"ℹ️ No route found via {travel_mode} for Origin: {origin_coords}, Dest: {dest_coords}. Result: {result}")
            return None, None

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {travel_mode}: {http_err} - Response: {response.text if 'response' in locals() else 'No response object'}")
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Request failed for {travel_mode}: {e}")
        return None, None
    except ValueError as e: # Catches JSON decoding errors
        print(f"⚠️ JSON decoding failed for {travel_mode}: {e} - Response: {response.text if 'response' in locals() else 'No response object'}")
        return None, None

# --- Main loop to process data ---
results = []
travel_modes_to_try = ["DRIVE", "TWO_WHEELER", "BICYCLE", "WALK"]

print(f"Starting API calls for {len(data)} rows, trying modes: {', '.join(travel_modes_to_try)}...")

for idx, row in data.iterrows():
    print(f"Processing row {idx + 1}/{len(data)}...")

    try:
        origin_lat = row['centroids_lat']
        origin_lon = row['centroids_lng']
        dest_lat = row['HF_Latitude']
        dest_lon = row['HF_Longitude']
        cell_id = row.get('cell_id', f"MISSING_CELL_ID_IDX_{idx}")
        city = row.get('city_x', "Unknown City")
    except KeyError as e:
        print(f"⚠️ Critical column missing in row {idx}: {e}. Skipping this row.")
        results.append({
            "cell_id": f"ERROR_MISSING_COLUMN_IDX_{idx}", "city": "N/A",
            "origin_lat": None, "origin_lng": None, "dest_lat": None, "dest_lng": None,
            "travel_time_min": "Error: Missing Column Data",
            "distance_meters": "Error: Missing Column Data",
            "travel_mode": "N/A" # New column
        })
        continue

    final_travel_time = "Error: No route found for any mode"
    final_distance = "Error: No route found for any mode"
    final_mode_used = "N/A"
    found_route_for_pair = False

    if pd.isna(origin_lat) or pd.isna(origin_lon) or \
       pd.isna(dest_lat) or pd.isna(dest_lon):
        print(f"⚠️ Skipping row {idx+1} (cell_id: {cell_id}) due to missing/NaN coordinates.")
        final_travel_time, final_distance, final_mode_used = "Error: Missing/NaN Coordinates", "Error: Missing/NaN Coordinates", "N/A"
    else:
        try:
            current_origin = {"latitude": float(origin_lat), "longitude": float(origin_lon)}
            current_destination = {"latitude": float(dest_lat), "longitude": float(dest_lon)}

            for mode_attempt in travel_modes_to_try:
                # print(f"  Trying mode: {mode_attempt} for row {idx+1} (CellID: {cell_id})...")
                travel_time_minutes, distance_val = get_travel_time_distance(
                    current_origin, current_destination, departure_timestamp, mode_attempt
                )

                if travel_time_minutes is not None and distance_val is not None:
                    final_travel_time = travel_time_minutes
                    final_distance = distance_val
                    final_mode_used = mode_attempt
                    found_route_for_pair = True
                    print(f"  ✅ Route found for row {idx+1} (CellID: {cell_id}) with {mode_attempt}: {final_travel_time:.2f} min, {final_distance} m")
                    break # Exit loop once a route is found for this pair

            if not found_route_for_pair:
                print(f"  ⚠️ No route found for any mode for row {idx+1} (CellID: {cell_id}).")
                # Values already set to "Error: No route found for any mode"

        except ValueError: # Error converting coordinates to float
            print(f"⚠️ Skipping row {idx+1} (cell_id: {cell_id}) due to invalid coordinate format.")
            final_travel_time, final_distance, final_mode_used = "Error: Invalid Coordinate Format", "Error: Invalid Coordinate Format", "N/A"

    results.append({
        "cell_id": cell_id,
        "city": city,
        "origin_lat": origin_lat,
        "origin_lng": origin_lon,
        "dest_lat": dest_lat,
        "dest_lng": dest_lon,
        "travel_time_min": final_travel_time,
        "distance_meters": final_distance,
        "travel_mode": final_mode_used
    })

    time.sleep(0.1) # Be respectful to the API rate limits

print("\nAPI calls completed.")

Starting API calls for 124 rows, trying modes: DRIVE, TWO_WHEELER, BICYCLE, WALK...
Processing row 1/124...
  ⚠️ No route found for any mode for row 1 (CellID: 150558.0).
Processing row 2/124...
  ⚠️ No route found for any mode for row 2 (CellID: 150974.0).
Processing row 3/124...
  ⚠️ No route found for any mode for row 3 (CellID: 151805.0).
Processing row 4/124...
  ⚠️ No route found for any mode for row 4 (CellID: 152218.0).
Processing row 5/124...
  ⚠️ No route found for any mode for row 5 (CellID: 152634.0).
Processing row 6/124...
  ⚠️ No route found for any mode for row 6 (CellID: 152635.0).
Processing row 7/124...
  ⚠️ No route found for any mode for row 7 (CellID: 152636.0).
Processing row 8/124...
  ⚠️ No route found for any mode for row 8 (CellID: 152637.0).
Processing row 9/124...
  ⚠️ No route found for any mode for row 9 (CellID: 153050.0).
Processing row 10/124...
  ⚠️ No route found for any mode for row 10 (CellID: 153052.0).
Processing row 11/124...
  ⚠️ No route found

#Display and Save the Result

In [ ]:
if results:
    results_df = pd.DataFrame(results)
    print("\nFirst 5 results:")
    print(results_df.head())
    try:
      results_df.to_csv('travel_times_output_multi_mode.csv', index=False, encoding='utf-8')
      print("\nResults saved to 'travel_times_output_multi_mode.csv'")
    except Exception as e:
      print(f"Error saving results to CSV: {e}")
else:
    print("No results were processed or generated.")


First 5 results:
    cell_id    city  origin_lat  origin_lng  dest_lat    dest_lng  \
0  150558.0  Jember   -8.480965  113.714199 -8.348291  113.608239   
1  150974.0  Jember   -8.476465  113.721993 -8.348291  113.608239   
2  151805.0  Jember   -8.467465  113.737582 -8.348291  113.608239   
3  152218.0  Jember   -8.444965  113.745376 -8.348291  113.608239   
4  152634.0  Jember   -8.440465  113.753170 -8.348291  113.608239   

                      travel_time_min                     distance_meters  \
0  Error: No route found for any mode  Error: No route found for any mode   
1  Error: No route found for any mode  Error: No route found for any mode   
2  Error: No route found for any mode  Error: No route found for any mode   
3  Error: No route found for any mode  Error: No route found for any mode   
4  Error: No route found for any mode  Error: No route found for any mode   

  travel_mode  
0         N/A  
1         N/A  
2         N/A  
3         N/A  
4         N/A  

Results